In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

In [4]:
spark = SparkSession.builder \
    .appName("Simple Example") \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
# Sample data
data = [("Hello Spark world",), ("Welcome to PySpark",), ("Spark is powerful",)]
df = spark.createDataFrame(data, ["sentence"])

# Process data
words_df = df.select(explode(split(col("sentence"), " ")).alias("word"))
word_counts = words_df.groupBy("word").count()

# Show results
word_counts.show()